### Set standard imports

In [ ]:
import matplotlib,lmfit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import KiMoPack.plot_func as pf
from importlib import reload
reload(pf)
import import_library as il
reload(il)


In [2]:
ta=pf.TA('gui',conversion_function=il.cor_streak_lund)
#ta=pf.TA('recent',conversion_function=il.cor_streak_lund)
#ta=pf.TA('FeCM02-266nm-4mw-QB390-t6-G63-w450-s150-556ms-E100.dat',conversion_function=il.streak_Lund,path='Data') #use 

In [ ]:
# Coarse adjustment
ta.intensity_range=[0,ta.ds.mean().max()]
if 0: # set to 1 for adjusting the zero point with a GUI
    %matplotlib tk
    ta.timelimits=[ta.ds.index.values.min(),ta.ds.index.values.max()]
    ta.Cor_Chirp(shown_window=ta.timelimits,just_shift=True,cmap=pf.cm.jet)
    if 0: # set to 1 for fine adjustment
        shown_window=np.array([-50,50])+ta.fitcoeff[-1]
        ta.Man_Chirp(shown_window=shown_window,just_shift=True,cmap=pf.cm.jet)
else: #use a manual plot to select t0
    %matplotlib inline
    pf.halfsize=True
    pf.changefonts()
    ta.Plot_RAW([0,1])
    t0=206
    ta.Cor_Chirp(fitcoeff=[0,0,0,0,t0])

In [ ]:
plt.close('all')
#ta.Background(uplimit=5,lowlimit=-100)
#ta.rel_wave=np.arange(480,560,10)#[400,410,420,460,480]
#ta.bordercut=[470,560]
#ta.rel_time=[-10,10,30,100,200,300,1000,1500]
#ta.timelimits=[1,1810]
#ta.wave_nm_bin=None
#ta.wavelength_bin=10
#ta.log_scale=False
#ta.intensity_range=[-2e3,3e4]
ta.Plot_RAW(range(3),scale_type='log')

### Perform fast fitting using independent exponential decay (first order)

In [ ]:
plt.close('all')                       # Close old spectra
ta.mod='exponential'                   # Choose the model 
par=lmfit.Parameters()                 # create empty parameter object
par.add('k0',value=1/10,vary=True,min=1/100)      # Add second rate component
par.add('k1',value=1/500,vary=True)   # Add one rate component
par.add('t0',value=0.28,vary=True) # Allow the arrival time to adjust (for all)
par.add('resolution',value=67,vary=True)       # Allow the instrument response to adjust (for all)
#par.add('infinite')                      # Keyword for an non decaying component
par.add('background')                    # Keyword to fit the Background 
ta.par=par                               # write parameter object 
ta.Fit_Global(fit_chirp=False)           # trigger fitting

ta.error_matrix_amplification=1
ta.Plot_fit_output([0,1,2,4])            # plot the fit output

In [ ]:
pf.halfsize=False
pf.changefonts()
ta.Save_Powerpoint()
pf.halfsize=True
pf.changefonts()

### Perform Error analysis and calculate confidence interval

In [ ]:
ta.par=ta.par_fit #write the best results back as starting parameter
ta.Fit_Global(confidence_level=0.66)

In [ ]:
plt.close('all')
ta.mod='consecutive'             # very quick fit using independent exponential followed by one round of sequential
ta.par=ta.par_fit                # copy fitted parameter into a new fit (e.g. different model)
ta.Fit_Global()   # The iterative chrip is best optimised using 'exponential'
ta.Plot_fit_output([0,1,2,4])     # plot the fit output